In [2]:
pip install schedule

  Using cached schedule-1.1.0-py2.py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Initial imports
import pandas as pd
import json
import alpaca_trade_api as tradeapi
import schedule, time
from pandas.tseries.offsets import BMonthEnd
import numpy as np
from datetime import datetime

In [11]:
# Set Alpaca API key and secret
alpaca_api_key = 'PKE9WE723IQ4PLSOUZVQ'
alpaca_secret_key = 'Vsj2SkscFm0wYE2PTyvGL9f4YaWvgKRMh4WL9tG5'

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    "https://paper-api.alpaca.markets"
)

In [31]:
def exec_trade():
    now = pd.Timestamp('now')
    print(now)
    print(type(now))
    account = api.get_account()
    cash = int(float(account.cash))
    offset = BMonthEnd()
    last_day=offset.rollforward(now)
    barset = api.get_barset('SPY', 'day', limit=200)
    SPY_closes = [bar.c for bar in barset["SPY"][:]]
    SMA = np.mean(SPY_closes)
    current_price = int(SPY_closes[-1])
    positions = api.list_positions()
    portfolio = [position.symbol for position in positions]
    open_position = False
    spy_qty = 0
    if [position for position in api.list_positions() if position.symbol=="SPY"]:
        spy_qty= int([position.qty for position in api.list_positions() if position.symbol=="SPY"][0])
        
    if "SPY" in portfolio:
        open_position=True
    if(now.date()==last_day):
        if((current_price > SMA) and (open_position)):
            api.submit_order(symbol="SPY", qty = int(cash/current_price), type="market", side="buy", time_in_force="gtc")
        elif((SMA>current_price) and (not open_position)):
            api.submit_order(symbol="SPY", qty=spy_qty, type="market", side="sell", time_in_force="gtc")
    

In [32]:
exec_trade()

2021-07-11 15:47:11.518647
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


C:\Users\rhnil\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


In [33]:
def running_print():
    print("Hello your bot is running")

schedule.every(10).seconds.do(running_print)
schedule.every(15).seconds.do(exec_trade)

Every 15 seconds do exec_trade() (last run: [never], next run: 2021-07-11 15:47:34)

In [17]:
while True:
    schedule.run_pending()
    time.sleep(1)

AttributeError: 'datetime.date' object has no attribute 'date'